In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cols = {'total', '1/8_salary', '1/4_salary', '1/2_salary',
       '1_salary', '2_salary', '3_salary', '5_salary', '10_salary'}

f = lambda col: lambda x: pd.to_numeric(x[col], errors='coerce').fillna(0)

col_clean = {col: f(col)
             for col in cols}

df = (pd.read_csv('/kaggle/input/rent-census-for-brazilian-municipalities/aluguel_municipio_br.csv')
        .assign(**col_clean)
     )


MINIMUM_SALARY = 1050 # BRL

VALUE_MAP = {
    '1/8_salary': MINIMUM_SALARY * (1/8 + 0.0) / 2,
    '1/4_salary': MINIMUM_SALARY * (1/8  + 1/4) / 2,
    '1/2_salary': MINIMUM_SALARY * (1/4  + 1/2) / 2,
    '1_salary': MINIMUM_SALARY * (1  + 1/2) / 2,
    '2_salary': MINIMUM_SALARY * (2  + 1) / 2,
    '3_salary': MINIMUM_SALARY * (3  + 2) / 2,
    '5_salary': MINIMUM_SALARY * (5  + 3) / 2,
    '10_salary': MINIMUM_SALARY * (10  + 5) / 2,
}

VALUE_MAP = {k + "_value": v for k, v in VALUE_MAP.items()}

VALUE_COLS = set(VALUE_MAP.keys()) - {'total'}

f = lambda col: lambda df: df[col[:-6]] * VALUE_MAP[col]
g = lambda df: df[VALUE_COLS].sum(axis=1)
h = lambda df: df.rent_total / df.total
k = lambda df: df[cols - {'total'}].median(axis=1)
VALUE_MAPS = {col: f(col) for col in VALUE_COLS}
df = (df.assign(**VALUE_MAPS)
        .assign(rent_total=g)
        .assign(average_rent=h))

In [ ]:
df.describe()

In [ ]:
df.sort_values(by='average_rent')

In [ ]:
(df.average_rent * df.total / df.total.sum()).sum()

In [ ]:
high_rents = {'10_salary', '5_salary', '3_salary', '2_salary'}
high_rents_value = {e + "_value" for e in high_rents}

print(f"Share of rents above 2 salary (count): {df[high_rents].sum().sum() / df.total.sum()}")
print(f"Share of rents above 2 salary (value): {df[high_rents_value].sum().sum() / df.rent_total.sum()}")

In [ ]:
high_rents = {'1/8_salary', '1/4_salary', '1/2_salary', '1_salary'}
high_rents_value = {e + "_value" for e in high_rents}

print(f"Share of rents above 2 salary (count): {df[high_rents].sum().sum() / df.total.sum()}")
print(f"Share of rents above 2 salary (value): {df[high_rents_value].sum().sum() / df.rent_total.sum()}")

In [ ]:
import requests as req
import json 

# https://github.com/tbrugz/geodata-br
GEOJSON_URI = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-100-mun.json'

r = req.get(GEOJSON_URI)
geo_json = json.loads(r.content)


In [ ]:
fig = px.choropleth(df, 
                    geojson=geo_json,
                    hover_name='city',
                    locations='city_id',
                    featureidkey='properties.id',
                    color='average_rent',
                    color_continuous_scale='RdYlGn_r',
                    width=1800,
                    height=1000
                  )
fig.update_geos(fitbounds="locations")
fig.update_traces(marker_line_width=0)
fig.show()

In [ ]:
5 + 5